In [2]:
import tensorflow as tf
from keras import models, layers
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
zip_path = '/content/drive/MyDrive/chest_x_ray_aug.zip'

# Specify the path to extract the files to
extract_path = '/content/drive/MyDrive/'

# Open the zip file
with ZipFile(zip_path, 'r') as zip_ref:
    # Extract all files to the specified path
    zip_ref.extractall(extract_path)

# Print a success message
print('Zip file extracted successfully!')

Zip file extracted successfully!


In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS=25

In [ ]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape= (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
)

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# for layer in conv_base.layers[:15]:
#     layer.trainable = False

# for layer in conv_base.layers:
#     print(layer.name, layer.trainable)

for layer in conv_base.layers:
    layer.trainable = False

for layer in conv_base.layers:
    print(layer.name, layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model = models.Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 64)                1605696   
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 16328962 (62.29 MB)
Trainable params: 1614274 (6.16 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [8]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    r"/content/drive/MyDrive/chest_x_ray_aug/train",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        r"/content/drive/MyDrive/chest_x_ray_aug/valid",
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/chest_x_ray_aug/train'

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

In [ ]:
model_version=4
model.save(f"/content/drive/MyDrive/models_vgg16/{model_version}")

In [3]:
from email.mime import image
from pyexpat import model
import tensorflow as tf
from keras import models, layers
import matplotlib.pyplot as plt
import cv2
import numpy as np

CLASS_NAMES = ['NORMAL', 'PNEUMONIA']

MODEL = tf.keras.models.load_model(r"/content/drive/MyDrive/models_vgg16/3")

image = cv2.imread(r"/content/drive/MyDrive/chest_x_ray_aug/train/PNEUMONIA/download.jpg")

# print(image.shape)

resized_image = cv2.resize(image, (224, 224))

# print(resized_image.shape)

img_array = tf.keras.preprocessing.image.img_to_array(resized_image)
img_array = tf.expand_dims(img_array, 0) # Create a batch
img_array = img_array / 255.0

predictions = MODEL.predict(img_array)


predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
# confidence = np.max(predictions[0])
confidence = round(100 * (np.max(predictions[0])), 2)

print(f"Prediction: {predicted_class} \nAccurecy: {confidence}%")


1/1 [==============================] - 3s 3s/step
Prediction: NORMAL 
Accurecy: 96.78%


In [ ]:
from keras import models, layers
MODEL = tf.keras.models.load_model(r"/content/drive/MyDrive/models_vgg16/3")

In [4]:
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import numpy as np

# Load the pre-trained model
model = tf.keras.models.load_model(r"D:\Deep Learning & ML\pneumonia_detection\Project_zip\Models\models_vgg16\3")

# Generate predictions on the validation data
test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
    r"D:\Deep Learning & ML\pneumonia_detection\training\chest_x_ray_aug\valid",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # Important: do not shuffle for evaluation
)

# Get true labels and predicted labels
Y_true = validation_generator.classes
Y_pred_prob = model.predict(validation_generator)
Y_pred = np.argmax(Y_pred_prob, axis=1)

# Compute precision, recall, and F1 score
precision = precision_score(Y_true, Y_pred)
recall = recall_score(Y_true, Y_pred)
f1 = f1_score(Y_true, Y_pred)

# Print the results
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Print detailed classification report
print(classification_report(Y_true, Y_pred, target_names=validation_generator.class_indices.keys()))

Found 1747 images belonging to 2 classes.
55/55 [==============================] - 606s 11s/step
Precision: 0.9950
Recall: 0.9434
F1 Score: 0.9685
              precision    recall  f1-score   support

      NORMAL       0.87      0.99      0.92       475
   PNEUMONIA       1.00      0.94      0.97      1272

    accuracy                           0.96      1747
   macro avg       0.93      0.97      0.95      1747
weighted avg       0.96      0.96      0.96      1747

